In [18]:
import schedule
import time
import datetime as dt

import requests                     # -> 웹 페이지 요청 모듈
import os                           # -> 운영체제 기능 접근 모듈(내장모듈)
import datetime as dt               # -> 날짜 처리 모듈(내장모듈)
from bs4 import BeautifulSoup       # -> 웹 페이지 소스코드 분석 모듈

import zipfile
import shutil

In [19]:
def collectNaverNews():
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"
    header_info = {'User-agent': user_agent, 'referer': None}
    header_info

    content_url = "https://news.naver.com/"

    # 뉴스 기사가 저장될 폴더이름 구성
    datetime = dt.datetime.now().strftime("%y%m%d_%H%M%S")
    dirname = "뉴스기사_%s" % datetime
    
    # 뉴스 기사에 대한 압축 파일 생성
    my_zip = zipfile.ZipFile("뉴스기사_%s.zip" % datetime, 'w') 

    # 뉴스기사를 텍스트 파일로 저장할 폴더 만들기
    if not os.path.exists(dirname):
        os.mkdir(dirname)

    r = requests.get(content_url, headers=header_info)

    # 결과 검사
    if r.status_code != 200:
        # 에러코드와 에러메시지를 문자열로 구성
        err_msg = "%d %s 에러가 발생했습니다." % (r.status_code, r.reason)
        # 강제로 에러를 발생시킨다.
        raise Exception(err_msg)

    r.encoding="euc-kr"
    html = r.text
    #html

    # 웹 페이지의 소스코드를 사용하여 HTML 분석 객체 생성
    soup = BeautifulSoup(html, 'html.parser')

    # CSS 선택자를 활용하여 가져오기를 원하는 부분 지정
    # -> 여러 개의 선택자를 사용해야 할 경우 콤마(,)로 구분
    selector = soup.select('.hdline_flick_item > .lnk_hdline_main_article, .hdline_article_tit > .lnk_hdline_article, .mtype_img a, .m2list a, .list_text_inner > a')

    # 결과 검사
    if len(selector) == 0:
        # 에러코드와 에러메시지를 문자열로 구성
        err_msg = "지정된 선택자에 대해서 인식된 구조가 없습니다."
        # 강제로 에러를 발생시킨다.
        raise Exception(err_msg)

    #print(len(selector))
    #print("-" * 50)
    #selector


    # 뉴스기사의 본문 URL을 저장할 리스트
    url_list = []

    # 리스트의 원소들에 대한 반복 처리
    for item in selector:
        # 각 원소(링크)에 속성들(attrs) 중에 href 속성이 있다면 그 값을 별도로 준비한 리스트에 추가
        if "href" in item.attrs:
            url_list.append(item['href'])

    url_list


    for i, v in enumerate(url_list):
        if content_url not in v:
            url = content_url + v
            url_list[i] = url

    url_list

    news_url_list = []

    for url in url_list:
        if "read.nhn" in url:
            news_url_list.append(url)

    news_url_list

    # URL 목록만큼 반복
    for i, url in enumerate(news_url_list):

        print("%d번째 뉴스기사 수집중... >> %s" % (i+1, url))

        #-------------------------------------------------
        # 1) 뉴스 상세 페이지에 접속하여 HTML 소스코드 가져오기
        #-------------------------------------------------
        r = requests.get(url, headers=header_info)

        # 접속에 실패했다면 다음 항목으로 반복문의 제어를 이동
        if r.status_code != 200:
            # 에러코드와 에러메시지를 문자열로 구성
            err_msg = "%d %s 에러가 발생했습니다." % (r.status_code, r.reason)
            print(err_msg)
            continue


        # 컨텐츠의 인코딩 형식 설정
        r.encoding = "euc-kr"

        #-------------------------------------------------
        # 2) 기사 영역 추출
        #-------------------------------------------------
        # HTML 코드 분석을 위한 객체 생성
        soup = BeautifulSoup(r.text, 'html.parser')

        #-------------------------------------------------
        # 3) 기사 영역에서 제목 추출
        #-------------------------------------------------
        title = soup.select("#articleTitle")
        #print(title)

        # 제목의 0번째 항목에서 텍스트를 공백 제거하고 추출
        title_str = title[0].text.strip()

        # 기사 제목에서 파일이름으로 사용할 수 없는 특수문자 제거
        title_str = title_str.replace("'", "")
        title_str = title_str.replace('"', "")
        title_str = title_str.replace("?", "")
        title_str = title_str.replace("/", "")
        title_str = title_str.replace(">", "")
        title_str = title_str.replace("<", "")

        # 둥근따옴표 제거
        # "`ㄴ`>한자" 입력후 표시되는 팝업에서 선택 가능함
        title_str = title_str.replace("‘", "") # `ㄴ`입력 후 `한자키` --> 1페이지에서 8번
        title_str = title_str.replace("’", "") # `ㄴ`입력 후 `한자키` --> 1페이지에서 9번
        title_str = title_str.replace("“", "") # `ㄴ`입력 후 `한자키` --> 2페이지에서 1번
        title_str = title_str.replace("”", "") # `ㄴ`입력 후 `한자키` --> 2페이지에서 2번

        print(title_str)

        #-------------------------------------------------
        # 4) 기사 영역에서 내용 추출
        #-------------------------------------------------
        # 본문 가져오기
        article = soup.select('#articleBodyContents')
        article_item = article[0]

        # 불필요한 태그 제거, 치환
        for target in article_item.find_all(['script', 'a', 'span']):
            target.extract()

        for target in article_item.find_all('br'):
            target.replace_with("\n")

        # 본문 텍스트 추출
        article_str = article_item.text.strip()

        #-------------------------------------------------
        # 5) 제목과 내용을 텍스트 파일로 저장
        #-------------------------------------------------
        # 제목과 내용이 모두 존재한다면?
        if title_str and article_str:
            # 기사 제목을 파일명으로 지정하여 내용을 텍스트로 저장한다.
            fname = dirname + "/" + title_str + ".txt"
            with open(fname, 'w', encoding='utf-8') as f:
                f.write(article_str)
                print(" >> 파일저장 성공: " + fname)
                
            # 저장된 파일을 압축파일에 추가
            my_zip.write(fname)
            
    # 압축 파일 닫기
    my_zip.close()

    # 폴더와 텍스트 파일은 삭제
    shutil.rmtree(dirname)

In [20]:
schedule.every(1).minutes.do(collectNaverNews)

Every 1 minute do collectNaverNews() (last run: [never], next run: 2021-05-25 13:35:03)

In [21]:
while True:    
    # 현재 시간에 동작해야 할 함수들을 찾아 구동함
    schedule.run_pending()
        
    # 1초 대기
    time.sleep(1)

157
--------------------------------------------------
1번째 뉴스기사 수집중... >> https://news.naver.com//main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=008&aid=0004592035
월가 비트코인 긍정론자 한번 더 폭락하면 매수 기회
 >> 파일저장 성공: 뉴스기사_210525_133404/월가 비트코인 긍정론자 한번 더 폭락하면 매수 기회.txt
2번째 뉴스기사 수집중... >> https://news.naver.com//main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=055&aid=0000896332
야3당, 세종시 특공 국정조사 요구서 공동 제출…부당이득 환수


FileNotFoundError: [Errno 2] No such file or directory: '뉴스기사_210525_133404/야3당, 세종시 특공 국정조사 요구서 공동 제출…부당이득 환수.txt'